<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

In [66]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [67]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [68]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [69]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [70]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [71]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [72]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-02 10:55:24
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.35 C
-------------------
Today PnL: -1.78 L (-1.3%)
Current PnL: -34.81 L (-22.2%)
CY Booked + Current PnL: -20.07 L (-12.8%)
-------------------
Total profit:  79.18 K
Total loss:  -35.60 L
-------------------
Total Booked + Current PnL: 6.91 L (5.33%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.94%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 1.00 C (74.48%)
Deployed:  1.30 C
Current:  1.35 C
CAGR/XIRR %: 1.92%


In [73]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,484.83,-5.62,2.30,16.55,19.23,25474.0,3460.0,153920.0,19.88,48.0,L-LC,12.43,182.0,0.14,1.14,24.06,XY25,ATH,MINING
77,TTKPRESTIG,770.00,-1.80,-26.22,35.66,0.09,26515.0,-26421.0,74356.0,72.68,34.0,M-SC,6.48,253.0,-1.00,0.55,0.00,OX40N,NTT,DURABLES
50,MASFIN,398.61,-5.55,-7.18,31.49,22.05,28639.0,-7035.0,90945.0,-20.05,43.0,H-SC,3.40,168.0,-0.25,0.68,32.20,XR,ATH,FINANCE
33,HCLTECH,1908.19,-2.70,8.71,13.91,23.83,36584.0,21082.0,263006.0,13.61,51.0,X-LC,7.30,13.0,0.58,1.95,26.18,X40,ATH,IT
51,MEDANTA,1486.00,6.37,-8.15,35.09,24.08,40915.0,-10350.0,116600.0,-15.60,46.0,X-SC,9.47,91.0,-0.25,0.87,6.80,XY24,NTT,HEALTHCARE


In [74]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,MEDANTA,1486.0,6.37,-8.15,35.09,24.08,40915.0,-10350.0,116600.0,-15.60,46.0,X-SC,9.47,91.0,-0.25,0.87,6.80,XY24,NTT,HEALTHCARE
68,STARHEALTH,761.0,4.91,-8.55,61.02,47.26,156879.0,-24029.0,257094.0,24.79,65.0,H-SC,3.32,174.0,-0.15,1.91,37.74,XY24,NTT,INSURANCE
15,CAMPUS,393.0,4.83,-11.99,51.45,33.28,75969.0,-20119.0,147656.0,-29.13,54.0,M-SC,5.77,221.0,-0.26,1.10,14.44,XY24,NTT,FOOTWEAR
2,ABBOTINDIA,35195.0,3.17,-7.81,28.07,18.07,46282.0,-13975.0,164880.0,-21.05,44.0,X-MC,8.75,64.0,-0.30,1.22,7.26,X40,ATH,PHARMA
45,JCHAC,2282.0,2.54,-39.92,66.44,-0.01,51014.0,-51023.0,76782.0,15617.39,44.0,M-SC,15.23,235.0,-1.00,0.57,6.45,OX40N,BTT,AC


In [75]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,3033.00,-9.50,-5.84,31.90,24.20,91693.0,-17812.0,287438.0,34.21,38.0,X-SC,2.52,99.0,-0.19,2.13,20.38,X40N,NTT,FINANCE
29,GREENPANEL,537.00,-6.06,-42.64,149.54,43.14,157044.0,-78060.0,105018.0,135.14,43.0,M-SC,17.39,231.0,-0.50,0.78,2.70,XY24,NTT,MISC
46,JIOFIN,387.00,-6.03,-23.08,61.82,24.48,127145.0,-61696.0,205669.0,-24.33,17.0,X-LC,7.67,37.0,-0.49,1.53,19.21,XY24,BTT,FINANCE
18,COALINDIA,484.83,-5.62,2.30,16.55,19.23,25474.0,3460.0,153920.0,19.88,48.0,L-LC,12.43,182.0,0.14,1.14,24.06,XY25,ATH,MINING
38,IEX,219.00,-5.55,-18.73,82.88,48.63,134681.0,-37453.0,162501.0,-45.96,30.0,H-SC,14.71,137.0,-0.28,1.21,0.00,XR,NTT,MISC


In [76]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.0,-1.41,-0.98,47.01,45.57,121782.0,-2563.0,259056.0,-36.4,64.0,H-MC,6.9,121.0,-0.02,1.92,15.04,AR,NTT,ELECTRICAL


In [77]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1641.55,-0.28,-10.23,131.60,107.91,174431.0,-15098.0,132546.0,-27.04,37.0,M-SC,10.84,220.0,-0.09,0.98,0.00,OX40N,ATH,CABLES
72,TATAELXSI,9161.00,1.06,-22.46,70.28,32.03,71840.0,-29615.0,102220.0,-16.04,48.0,H-SC,6.52,157.0,-0.41,0.76,14.50,OX40N,NTT,IT
66,SIS,528.00,2.25,-23.33,58.15,21.26,49504.0,-25900.0,85132.0,2009.33,54.0,H-SC,4.82,163.0,-0.52,0.63,15.02,OX40N,NTT,MISC
48,KANSAINER,340.00,-0.77,-25.35,52.00,13.47,104682.0,-68355.0,201312.0,-69.28,39.0,H-SC,4.99,159.0,-0.65,1.50,2.36,XY24,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.80,-26.22,35.66,0.09,26515.0,-26421.0,74356.0,72.68,34.0,M-SC,6.48,253.0,-1.00,0.55,0.00,OX40N,NTT,DURABLES


In [78]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [79]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,484.83,-5.62,2.3,16.55,19.23,25474.0,3460.0,153920.0,19.88,48.0,L-LC,12.43,182.0,0.14,1.14,24.06,XY25,ATH,MINING


In [80]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-1.41,-0.98,47.01,45.57,121782.0,-2563.0,259056.0,-36.40,64.0,H-MC,6.90,121.0,-0.02,1.92,15.04,AR,NTT,ELECTRICAL
84,WIPRO,420.00,1.75,-1.13,74.24,72.27,124374.0,-1918.0,167530.0,-15.15,45.0,M-LC,2.56,101.0,-0.02,1.24,7.68,XR,NTT,IT
39,INDIAMART,4810.62,0.72,-5.85,119.56,106.72,138837.0,-7213.0,116123.0,-53.30,49.0,H-SC,9.06,138.0,-0.05,0.86,16.16,AR,ATH,MISC
26,FINCABLES,1641.55,-0.28,-10.23,131.60,107.91,174431.0,-15098.0,132546.0,-27.04,37.0,M-SC,10.84,220.0,-0.09,0.98,0.00,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,-2.52,-8.44,48.13,35.63,92359.0,-17680.0,191894.0,-21.17,37.0,H-MC,5.50,119.0,-0.19,1.43,6.83,AR,ATH,PHARMA


In [81]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,JIOFIN,387.00,-6.03,-23.08,61.82,24.48,127145.0,-61696.0,205669.0,-24.33,17.0,X-LC,7.67,37.0,-0.49,1.53,19.21,XY24,BTT,FINANCE
13,BERGEPAINT,680.00,-1.10,-18.44,48.54,21.14,89998.0,-41930.0,185409.0,-24.20,18.0,X-MC,7.34,75.0,-0.47,1.38,0.00,XY24,NTT,PAINTS
12,BATAINDIA,2096.00,-0.16,-44.88,145.28,35.20,103044.0,-57742.0,70928.0,-7.00,20.0,X-SC,19.88,93.0,-0.56,0.53,0.80,X40,NTT,FOOTWEAR
6,ASIANPAINT,3460.25,-1.00,-15.98,43.94,20.95,92952.0,-40225.0,211543.0,-16.80,23.0,X-LC,10.58,36.0,-0.43,1.57,13.97,X40,ATH,PAINTS
44,ITC,452.00,1.11,-21.04,40.31,10.78,76487.0,-50572.0,189746.0,-53.00,24.0,X-LC,17.74,1.0,-0.66,1.41,1.11,X40,NTT,FMCG


In [82]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4311.59,1.09,-13.16,36.52,18.55,119947.0,-49795.0,328442.0,-25.24,50.0,X-LC,0.55,3.0,-0.42,2.44,11.74,X40,ATH,IT
81,VBL,671.64,-0.12,-5.77,43.97,35.67,130867.0,-18215.0,297627.0,-17.02,38.0,X-LC,1.83,5.0,-0.14,2.21,7.29,X40N,ATH,FMCG
34,HINDUNILVR,2922.00,-2.28,-9.79,26.00,13.67,60294.0,-25163.0,231900.0,-16.61,42.0,X-LC,2.25,9.0,-0.42,1.72,9.48,XY25,NTT,FMCG
5,ANGELONE,3033.00,-9.50,-5.84,31.90,24.20,91693.0,-17812.0,287438.0,34.21,38.0,X-SC,2.52,99.0,-0.19,2.13,20.38,X40N,NTT,FINANCE
36,ICICIGI,2252.93,-0.31,-3.59,23.79,19.34,45896.0,-7189.0,192920.0,-23.80,35.0,X-MC,2.74,71.0,-0.16,1.43,11.77,X40,ATH,INSURANCE


In [83]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-3.84,-32.17,142.01,64.15,229204.0,-76551.0,161400.0,-60.24,30.0,X-SC,7.28,84.0,-0.33,1.20,0.00,XY24,BTT,CEMENT
10,BAJAJHLDNG,14451.52,-1.89,-5.90,37.32,29.22,66768.0,-11217.0,178908.0,-8.67,36.0,X-LC,13.85,29.0,-0.17,1.33,0.00,X40,ATH,FINANCE
13,BERGEPAINT,680.00,-1.10,-18.44,48.54,21.14,89998.0,-41930.0,185409.0,-24.20,18.0,X-MC,7.34,75.0,-0.47,1.38,0.00,XY24,NTT,PAINTS
32,HAVELLS,2062.85,-2.09,-19.44,63.95,32.08,168970.0,-63769.0,264222.0,-23.56,25.0,X-MC,7.87,69.0,-0.38,1.96,0.00,X40,ATH,ELECTRICAL
53,PGHH,17907.65,0.06,-11.87,52.26,34.19,98341.0,-25344.0,188176.0,-38.01,28.0,X-MC,7.36,60.0,-0.26,1.40,0.06,X40,ATH,FMCG


In [84]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.34,-31.13,107.41,42.84,48087.0,-20236.0,44770.0,-54.82,47.0,X-SC,21.60,83.0,-0.42,0.33,5.24,XY24,NTT,MISC
12,BATAINDIA,2096.00,-0.16,-44.88,145.28,35.20,103044.0,-57742.0,70928.0,-7.00,20.0,X-SC,19.88,93.0,-0.56,0.53,0.80,X40,NTT,FOOTWEAR
58,RELAXO,1176.00,-3.72,-51.22,212.23,52.31,150280.0,-74350.0,70810.0,-48.02,45.0,X-SC,11.45,92.0,-0.49,0.53,5.30,X40N,NTT,FOOTWEAR
51,MEDANTA,1486.00,6.37,-8.15,35.09,24.08,40915.0,-10350.0,116600.0,-15.60,46.0,X-SC,9.47,91.0,-0.25,0.87,6.80,XY24,NTT,HEALTHCARE
35,HONAUT,58357.33,0.84,-20.02,79.20,43.32,103166.0,-32612.0,130260.0,-31.26,45.0,X-SC,9.02,90.0,-0.32,0.97,5.76,X40N,ATH,ELECTRICAL


In [85]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,452.00,1.11,-21.04,40.31,10.78,76487.0,-50572.0,189746.0,-53.00,24.0,X-LC,17.74,1.0,-0.66,1.41,1.11,X40,NTT,FMCG
75,TMPV,600.00,1.27,-27.34,69.25,22.98,113663.0,-61764.0,164134.0,-24.30,52.0,X-LC,4.56,2.0,-0.54,1.22,4.93,XY24,NTT,AUTO
73,TCS,4311.59,1.09,-13.16,36.52,18.55,119947.0,-49795.0,328442.0,-25.24,50.0,X-LC,0.55,3.0,-0.42,2.44,11.74,X40,ATH,IT
81,VBL,671.64,-0.12,-5.77,43.97,35.67,130867.0,-18215.0,297627.0,-17.02,38.0,X-LC,1.83,5.0,-0.14,2.21,7.29,X40N,ATH,FMCG
42,INFY,1972.00,-0.27,11.41,20.50,34.24,71458.0,35686.0,348574.0,-13.14,50.0,X-LC,7.14,6.0,0.50,2.59,20.63,X40,NTT,IT


In [86]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1800.00,-0.49,-34.00,101.34,32.89,51641.0,-26248.0,50958.0,-729.38,50.0,L-MC,11.29,259.0,-0.51,0.38,40.33,XR,NTT,BANKS
14,BSOFT,831.70,1.62,-24.48,99.66,50.79,105859.0,-34433.0,106220.0,-2.98,50.0,H-SC,5.90,171.0,-0.33,0.79,23.68,XR,ATH,IT
18,COALINDIA,484.83,-5.62,2.30,16.55,19.23,25474.0,3460.0,153920.0,19.88,48.0,L-LC,12.43,182.0,0.14,1.14,24.06,XY25,ATH,MINING
68,STARHEALTH,761.00,4.91,-8.55,61.02,47.26,156879.0,-24029.0,257094.0,24.79,65.0,H-SC,3.32,174.0,-0.15,1.91,37.74,XY24,NTT,INSURANCE
33,HCLTECH,1908.19,-2.70,8.71,13.91,23.83,36584.0,21082.0,263006.0,13.61,51.0,X-LC,7.30,13.0,0.58,1.95,26.18,X40,ATH,IT


In [87]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SATIN,274.00,0.15,-14.39,77.35,51.83,182964.0,-39744.0,236540.0,-23.78,61.0,H-SC,3.11,148.0,-0.22,1.76,15.23,XY24,NTT,FINANCE
27,GILLETTE,11206.78,-1.51,2.41,29.30,32.42,76183.0,6114.0,260010.0,-15.80,63.0,X-SC,12.67,87.0,0.08,1.93,17.87,X40,ATH,FMCG
1,ABB,7934.00,-1.41,-0.98,47.01,45.57,121782.0,-2563.0,259056.0,-36.40,64.0,H-MC,6.90,121.0,-0.02,1.92,15.04,AR,NTT,ELECTRICAL
33,HCLTECH,1908.19,-2.70,8.71,13.91,23.83,36584.0,21082.0,263006.0,13.61,51.0,X-LC,7.30,13.0,0.58,1.95,26.18,X40,ATH,IT
68,STARHEALTH,761.00,4.91,-8.55,61.02,47.26,156879.0,-24029.0,257094.0,24.79,65.0,H-SC,3.32,174.0,-0.15,1.91,37.74,XY24,NTT,INSURANCE


In [88]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.84
1,25,44.90
2,50,76.83


In [89]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
SC,45.03
MC,30.60
LC,24.39


In [90]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Criteria,
XY24,26.38
X40,24.16
X40N,14.81
AR,8.89
XR,8.70
XY25,8.66
OX40N,7.49
SR,0.93


In [91]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
X-MC,23.95
H-SC,23.15
X-LC,20.90
M-SC,11.00
X-SC,10.14
H-MC,4.80
M-MC,1.47
M-LC,1.24
L-LC,1.14


In [92]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.30,-14.91,54.08
FINANCE,13.10,-18.54,65.21
IT,13.02,-21.88,82.77
MISC,6.88,-40.01,97.28
ELECTRICAL,5.96,-16.86,59.49
PAINTS,5.39,-27.49,46.43
INSURANCE,4.88,-3.26,38.60
PHARMA,4.05,-7.81,41.75
BANKS,2.94,-35.11,116.98


In [93]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3421196.0,21
AR,1375145.0,10
XR,1320037.0,12
X40,1255746.0,15
X40N,1125735.0,10
OX40N,783334.0,10
XY25,454775.0,6
SR,295192.0,2


In [94]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3679578.0,24
X-MC,1807616.0,16
M-SC,1577852.0,15
X-LC,1076788.0,12
X-SC,933351.0,9
H-MC,431934.0,3
L-SC,191052.0,2
M-LC,124374.0,1
H-LC,83927.0,1


In [95]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1249862.0      6
           AR         946156.0      5
M-SC       XY24       898995.0      6
H-SC       XR         853897.0      7
X-MC       X40        612323.0      7
           XY24       495532.0      3
X-LC       X40        464196.0      6
X-MC       X40N       454230.0      4
X-SC       X40N       435918.0      4
M-SC       OX40N      353598.0      5
H-SC       OX40N      334471.0      4
X-SC       XY24       318206.0      3
H-SC       SR         295192.0      2
X-MC       XY25       245531.0      2
X-LC       XY24       240808.0      2
           X40N       235587.0      2
H-MC       XY24       217793.0      1
           AR         214141.0      2
M-SC       XR         194338.0      2
X-SC       X40        179227.0      2
X-LC       XY25       136197.0      2
M-SC       AR         130921.0      2
M-LC       XR         124374.0      1
L-SC       XR          95787.0      1
           OX40N       95265.0      1
H-LC       AR          83927.0      1
L-MC       XR          51641.0      1
M-MC       XY25        47573.0      1
L-LC       XY25        25474.0      1

In [96]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
